In [ ]:
# figure out how to run the model for inference.
# it needs to predict past data for tomorrows direction of forex stock
# we have trained the model in timeseries past data 
# let's try to make it work, I think what the model needs is just the today's data inorder to make the predictions. Or I am not sure if I should provide even the past data for inference.
# For now let's see and try to work with today's data for predicting tomorrow.

In [3]:
import psycopg2
import joblib
import pandas as pd
import numpy as np
import json

model = joblib.load("model_service/forex_model.pkl")
conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port=5433)
query = """SELECT * FROM eur_usd_final ORDER BY datetime DESC LIMIT 1;"""
input_df = pd.read_sql(query, conn)
conn.close()
print(input_df)

     id    datetime     open     high      low    close  close_ratio_2  \
0  5040  2026-01-05  1.17238  1.17251  1.16724  1.16871       0.998577   

   close_ratio_5  close_ratio_60  close_ratio_250  close_ratio_1000  
0       0.996652        1.003938         1.017809          1.075275  


/var/folders/5l/17wd4myj387b056cpgkkpxth0000gn/T/ipykernel_90381/1792178877.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  input_df = pd.read_sql(query, conn)


In [4]:
input_df

,id,datetime,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000
0,5040,2026-01-05,1.17238,1.17251,1.16724,1.16871,0.998577,0.996652,1.003938,1.017809,1.075275


In [5]:
predictors = input_df.columns.drop(["id", "datetime"])
predictors

Index(['open', 'high', 'low', 'close', 'close_ratio_2', 'close_ratio_5',
       'close_ratio_60', 'close_ratio_250', 'close_ratio_1000'],
      dtype='object')

In [6]:
input_df["prediction"] = model.predict(input_df[predictors])

In [5]:
input_df

,id,datetime,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000,prediction
0,5040,2026-01-05,1.17238,1.17251,1.16724,1.16871,0.998577,0.996652,1.003938,1.017809,1.075275,0


In [6]:
output = {"datetime": input_df["datetime"], "prediction": input_df["prediction"]}

In [7]:
output

{'datetime': 0    2026-01-05
 Name: datetime, dtype: object,
 'prediction': 0    0
 Name: prediction, dtype: int64}